## Your Model 🌱Garden🌱 Execution Environment

In this notebook, you'll:
1. Create the environment your model needs to run
2. Get your model running, debugging right here
3. Add your code and model to a Garden Pipeline
4. Publish!

Garden will take this notebook and build a container with it. When Garden executes your `@garden_entrypoint`, it will be like like you have just run all the cells of this notebook once. So you can install libraries with `!pip install` and your function can use those libraries. You can also define helper functions and constants to use in your `@garden_entrypoint`.

## Environment Set Up
Use this section to install and import any packages you need for your model environment. We've included what you'll need from Garden and other packages you'll need for a sklearn model.

In [ ]:
#Install any packages you need here

In [ ]:
#You need these from Garden. Keep them!
from garden_ai.model_connectors import HFConnector
from garden_ai import EntrypointMetadata, garden_entrypoint

In [ ]:
#Import any other packages you need here
import pandas as pd
import sklearn
import joblib

## Get Your Model Running
Now that you're environment is set up, get your model running! Be sure to include any additional functions (like pre and post procesing) that go with your model.

### Make Things Modular with Steps (optional)
To make your work more reproducible and interpretable, consider breaking your code into different functions (if applicable). For example, preprocessing code would be its own function, running the model would be its own separate function, etc.

These separate functions will be considered Steps in your Pipeline. Steps are a way to highlight the different stages of your workflow. Breaking the code down into steps helps others understand how your Pipeline works and what each part does. Each function will be considered its own distinct Step in your Pipeline.

The first Step in a Pipeline will always be the Entrypoint. If you decide not to make Steps, you will have an Entrypoint without a Pipeline that runs the one function you have specified.

In [ ]:
def preprocess(input_df: pd.DataFrame) -> pd.DataFrame:
    input_df.fillna(0, inplace=True)
    return input_df

### Model connectors

Model connectors let Garden import metadata about your model.
They also have a `stage` method that you can use to download your model weights.

In [ ]:
my_hugging_face_repo = HFConnector("garden-ai/sample_sklearn_model")

### Stage Your HuggingFace Model
This allows us get your model from HuggingFace and pull it into your Garden.

In [ ]:
def stage_model():
    model_binary = garden_ai.hugging_face_connector(
        repo_id = "", # add your huggingface repo ID
        filename = "" # add the filename
    )
    
    model = joblib.load(model_binary)
    
    return model

### Entrypoint metadata


To publish your function, Garden needs metadata so that other users can discover it.
Edit this EntrypointMetadata object to describe your function.


In [ ]:
my_entrypoint_meta = EntrypointMetadata(
    title="My Inference Function",
    description="Write a longer description here so that people know what your entrypoint does.",
    authors=["you", "your collaborator"],
    tags=["materials science", "your actual field"]
)

## Write your entrypoint function that will run remotely

Write a function that executes your model(s). Tag that function with the `@garden_entrypoint` decorator.

The `@garden_entrypoint` decorator makes this function available to run in your garden when you publish the notebook.
Download your model weights and call your model in this function.

In the decorator be sure to include:
- your entrypoint metadata,
- connectors for any models you're using,
- the DOI of the garden you want this entrypoint to be found in. (Check `garden-ai garden list` for the DOIs of your gardens.)

In [ ]:
@garden_entrypoint(metadata=my_entrypoint_meta,  model_connectors=[my_hugging_face_repo], garden_doi="10.23677/my-garden-doi")
def run_my_model(input_df: pd.DataFrame) -> pd.DataFrame:
    cleaned_df = preprocess(input_df)
    download_path = my_hugging_face_repo.stage()
    model = joblib.load(f"{download_path}/model.pkl")
    return model.predict(cleaned_df)

### Test your entrypoint function

Finally, make sure your `@garden_entrypoint` works!
When Garden makes a container from your notebook, it runs all the cells in order and saves the notebook. Then users invoke your `@garden_entrypoint` in the context of the notebook.

If you can hit "Kernel" -> "Restart and run all cells" and your test below works, your `@garden_entrypoint` will work in your garden!


In [ ]:
# Replace with input that is relevant for your garden_entrypoint
example_input = pd.DataFrame({
    'A': [1, 2, None, 4],
    'B': [None, 2, 3, 4],
    'C': [1, 2, 3, 4]
})
run_my_model(example_input)

## Register Your Pipeline with Garden
Now it's time to register your pipeline with Garden. If you have more than one function, you'll list them in the order you'd like them executed.

In [ ]:
@garden_ai.pipeline(name="") #name your pipeline

def pipeline(input_data)
    
    #example pipeline. Fill in with your own pipeline functions
    ready_data = preprocessing(input_data)
    model = (stage_model)
    
    return model.predict(ready_data)
                    